In [3]:
# Animating a text spinner with a thread.
import threading
import itertools
import time
import sys

class Signal: 
    go = True
    
def spin(msg, signal):
    write, flush = sys.stdout.write, sys.stdout.flush 
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg 
        write(status)
        flush()
        write('\x08' * len(status)) 
        time.sleep(.1)
        if not signal.go: 
            break
    write(' ' * len(status) + '\x08' * len(status))
    
def slow_function():
    # pretend waiting a long time for I/O 
    time.sleep(3)
    return 42

def supervisor():
    signal = Signal()
    spinner = threading.Thread(target=spin,args=('thinking!', signal)) 
    print('spinner object:', spinner)
    spinner.start()
    result = slow_function() 
    signal.go = False 
    spinner.join()
    return result

def main():
    result = supervisor() 
    print('Answer:', result)

main()

spinner object: <Thread(Thread-5, initial)>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking          Answer: 42


In [8]:
# Animating a text spinner with a coroutine.
import asyncio
import itertools
import sys

@asyncio.coroutine
def spin(msg):
    write, flush = sys.stdout.write, sys.stdout.flush 
    for char in itertools.cycle('|/-\\'):
        status = char + ' ' + msg 
        write(status)
        flush()
        write('\x08' * len(status)) 
        try:
            yield from asyncio.sleep(.1) 
        except asyncio.CancelledError:
            break
    write(' ' * len(status) + '\x08' * len(status))

@asyncio.coroutine
def slow_function():
    # pretend waiting a long time for I/O 
    yield from asyncio.sleep(3)
    return 42

@asyncio.coroutine
def supervisor():
    spinner = asyncio.Task(spin('thinking!')) 
    print('spinner object:', spinner)
    result = yield from slow_function() 
    spinner.cancel()
    return result


def main():
    loop = asyncio.get_event_loop()
    result = loop.run_until_complete(supervisor()) 
    loop.close()
    print('Answer:', result)

    
main()

RuntimeError: This event loop is already running

spinner object: <Task pending coro=<spin() running at <ipython-input-8-45c4f2ee5504>:6>>
| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking/ thinking- thinking\ thinking| thinking          